In [1]:
#TODO Automatizar la lectura de archivos con nombres genéricos, ocupa mucho espacio del notebook


In [2]:
import pandas as pd
import numpy as np
from get_gene_info import get_genome_info, get_genes_from_ranges

# Get deleted genes

Getting the genes that should be affected by the deleted ranges (Only for Delta 16 from MG1655 and all the W3110 strains as the others already report which genes are deleted)


#### MG1655

In [3]:

delta16 = pd.read_csv('../files/deleted_ranges/delta_16.csv',encoding='latin1')

genomeMG1655_path = '../files/genomes/MG1655.gb'
genomeMG1655 = get_genome_info(genomeMG1655_path)

delta_16_genes = get_genes_from_ranges(delta16, genomeMG1655)

delta_16_genes.to_csv('../files/deleted_genes/mapped_del_genes_Δ16.csv',header=True)

1170


In [4]:
## Get files with the lengths of the genes from de MG1655

In [5]:
## Las MDS y MS56 ya traían sus bnumber, no hubo necesidad de mapear
#MS56
genes_MS56 = list(pd.read_csv('../files/deleted_ranges/MS56.csv')['Locus_Tag'])

#MDS
MDS_line = pd.read_csv('../files/deleted_ranges/MDS42_69.csv')
MD12v2 = pd.read_csv('../files/deleted_ranges/MDS12_V2.csv')
MDS_line= MDS_line.set_index('bnumber')
genes_12_v2=list(set(MD12v2['MDS12']))
genes_42 = list(set(MDS_line['MDS42_deleted'][MDS_line['MDS42_deleted']!=0].index))
genes_69 = list(set(MDS_line['MDS69_deleted'][MDS_line['MDS69_deleted']!=0].index))

In [6]:
strains_wbnumber = {'MS56':genes_MS56,'MDS12':genes_12_v2, 'MDS42':genes_42, 'MDS69':genes_69}

In [7]:
genes_finales = {}

for strain in strains_wbnumber.keys():
    genes_finales[strain] = genomeMG1655.loc[genomeMG1655.Bnumber.isin(strains_wbnumber[strain]),['Bnumber', 'Gen', 'Length']]
    genes_finales[strain].to_csv('../files/deleted_genes/mapped_del_genes_'+strain+'.csv',header=True)

#### W3110

In [8]:
names_W3110 = ['MGF01', 'MGF02', 'DGF327', 'DGF298']

rangos = pd.read_csv('../files/deleted_ranges/MGF_line.csv',encoding='latin1')
genomeW3110_path = '../files/genomes/W3110.gb'
genomeW3110 = get_genome_info(genomeW3110_path)

strains = []

In [9]:
for name_W3110 in names_W3110:
    strains.append(rangos.loc[:,[name_W3110+'_L', name_W3110+'_R']].dropna())

In [10]:
genes_finales = {}

for strain in strains:
    nombre = strain.columns[0].split('_')[0]
    genes_finales[nombre] = get_genes_from_ranges(strain, genomeW3110)
    genes_finales[nombre] = genes_finales[nombre].drop_duplicates()
    genes_finales[nombre].to_csv('../files/deleted_genes/mapped_del_genes_'+strain+'.csv',header=True)    

1035
1176
1392
1727


# Proteome mappiing

In [11]:
proteomic_data = pd.read_excel("../files/proteome_Schmidt/Proteomic_data_fg.xlsx") #Datos en fg  de distintas condiciones
proteome_size = pd.read_excel("../files/proteome_Schmidt/Proteome_size.xlsx")

In [12]:
#Delta16
delta16 = pd.read_csv("../files/deleted_genes/mapped_del_genes_Δ16.csv")
delta16 = delta16.drop('Unnamed: 0', axis=1)

In [15]:
## W3110
cepas = {}

for name_W3110 in names_W3110:
    strain_df = pd.read_csv("../files/deleted_genes/mapped_del_genes_"+name_W3110+".csv")
    strain_df = strain_df.drop('Unnamed: 0', axis=1)
    cepas[name_W3110]=strain_df

In [16]:
matched_genes_cepas={}
for n in names_W3110:
    print(n)
    matched_genes_cepas[n]=proteomic_data[proteomic_data['Bnumber'].isin(cepas[n].Bnumber)].copy()
    gene_length  = cepas[n].set_index('Bnumber').loc[matched_genes_cepas[n].Bnumber,['Gen','Length']].values
    matched_genes_cepas[n].loc[:,['Gen NCBI', 'Length']] = gene_length
    
    # Reorganize columns
    
    first = list(matched_genes_cepas[n].columns[:2])
    second = list(matched_genes_cepas[n].columns[-2:])
    third = list(matched_genes_cepas[n].columns[2:-2])
    
    matched_genes_cepas[n] = matched_genes_cepas[n].loc[:,first+second+third]

MGF01
MGF02
DGF327
DGF298


In [17]:
prot_load = []
prot_perc = []
# mat_gen = []

for nombre in names_W3110:
    matched_genes = matched_genes_cepas[nombre]
    
    # Agregamos la carga proteomica de cada condición
    prot_load.append((matched_genes.sum()[4:26]).rename(nombre))

    # Agregamos el porcentaje de cada condición
    prot_perc.append((matched_genes.sum()[4:26]/proteome_size.iloc[:,1:]*100).rename(index={0:nombre}))



In [18]:
for n in range(len(prot_perc)):
    prot_perc[n] = prot_perc[n].loc[:,["LB","Glicerol + AA","42°C glucosa","Fructosa","pH6 glucosa","Glucosa","Estrés-osmótico glucosa","Xilosa","Quimiostato µ=0.5","Manosa","Glicerol","Glucosamina","Succinato","Fumarato","Piruvato","Quimiostato µ=0.35","Acetato","Galactosa","Quimiostato µ=0.20","Quimiostato µ=0.12","Fase estacionaria 1 día","Fase estacionaria 3 días"]]